In [1]:
import pandas as pd
import numpy as np
from itertools import compress

In [13]:

def tsv_to_dataframe(file_tsv, structure = "normal"):
    '''
    this function transforms Eurostat tsv file in pandas dataframe
    file_tsv: file name. It's work with tsv and compressed file "tsv.gz"
    '''
    
    if structure == "inverse":
        print("use tsv_to_dataframe_long instead tsv_to_dataframe")
    
    def clean_cells(x):
        '''This function transforms Eurostat Missing Values ": " in numpy missing values.
        Then clean Eurostat annotation "b, u, .."'''
        try:
            return float(x)
        except:
            try:
                return float(x.split(" ")[0])
            except:
                return np.nan
    # open the Eurostat TSV file 

    data = pd.read_csv(file_tsv, sep="\t")
    # Create a dataframe for values data
    data_clean = data
    # Clean data values with clean_cells function
    data_clean = data_clean.applymap(lambda x: clean_cells(x))
    # Drop column with variable name like "age,isced11,unit,sex,geo\time". It is the first column. we have a 
    # dataframe with only data values 
    data_clean.drop(data_clean.columns[0], axis = 1, inplace = True)
    # transform column with variable in multiple-columns  
    variabili = data[data.columns[0]].apply(lambda x: pd.Series(x.split(",")))
    variabili.columns = data.columns[0].split(",")
    # return cleaned dataframe in pandas dataframe
    return pd.concat([variabili, data_clean], axis = 1)

In [23]:
def tsv_to_dataframe_long(file_tsv, structure = "normal"):
    '''
    this function transforms Eurostat tsv file in pandas dataframe
    file_tsv: file name. It's work with tsv and compressed file "tsv.gz"
    structure: "normal", columns indicates time
    structure: "inverse", columns indicates Nuts / geo
    
    '''
    
    def clean_cells(x):
        '''This function transforms Eurostat Missing Values ": " in numpy missing values.
        Then clean Eurostat annotation "b, u, .."'''
        try:
            return float(x)
        except:
            try:
                return float(x.split(" ")[0])
            except:
                return np.nan
            
    def annotation(x):
            '''This function extracts Eurostat annotation "b, u, .."'''
            try:
                return x.split(" ")[1]
            except:
                return np.nan

    def columns_type(x):
            try:
                return int(x.strip())
            except:
                if x == "geo\\time":
                    return "var_" + "geo"
                else:
                    return "var_" + x
                
    
    def columns_type_inverse(columns):
        new_columns =[]
        sep = columns.get_loc("time\\geo")
        new_columns.extend(list(columns[:sep].map(lambda x: "var_" + x)))
        new_columns.append("var_time")
        new_columns.extend(list(columns[sep + 1:]))
        return new_columns
    
    # open the Eurostat TSV file 

    data = pd.read_csv(file_tsv, sep="\t")
    # Create a dataframe for values data
    data_clean = data
    # Clean data values with clean_cells function
    data_clean = data_clean.applymap(lambda x: x)
    # Drop column with variable name like "age,isced11,unit,sex,geo\time". It is the first column. we have a 
    # dataframe with only data values 
    data_clean.drop(data_clean.columns[0], axis = 1, inplace = True)
    # transform column with variable in multiple-columns  
    variabili = data[data.columns[0]].apply(lambda x: pd.Series(x.split(",")))
    variabili.columns = data.columns[0].split(",")
    # return cleaned dataframe in pandas dataframe
    result = pd.concat([variabili, data_clean], axis = 1)
    
    if structure == "normal":
        colonne  = list(map(lambda x: columns_type(x), list(result.columns.values)))
        result.columns = colonne
        colonne_var = result.columns.map(lambda x: str(x)[0:3] == "var")
        index = list(compress(result.columns, colonne_var))
        result = result.melt(id_vars = index)
        result["value_raw"] = result["value"]
        result["eurostat_annotation"] = result["value_raw"].apply(lambda x: annotation(x))
        result["value"] = result["value"].apply(clean_cells) 
    elif structure == "inverse":
        result.columns = columns_type_inverse(result.columns)
        colonne_var = result.columns.map(lambda x: str(x)[0:3] == "var")
        index = list(compress(result.columns, colonne_var))
        result = result.melt(id_vars = index, var_name = "geo")
        result["value_raw"] = result["value"]
        result["eurostat_annotation"] = result["value_raw"].apply(lambda x: annotation(x))
        result["value"] = result["value"].apply(clean_cells) 
        
    
    return result

In [11]:
#example 
#if __name__ == "__main__":
import urllib.request
import gzip

eurostat_link = "https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/demo_nsinrt.tsv.gz"
urllib.request.urlretrieve(eurostat_link , "file.tsv.gz")

('file.tsv.gz', <http.client.HTTPMessage at 0x7fc659c2cfd0>)

In [22]:
result = tsv_to_dataframe_long("file.tsv.gz", structure = "inverse")

KeyError: 'time\\geo'

In [6]:
result

NameError: name 'result' is not defined

In [16]:
columns_type_inverse(result.columns)

NameError: name 'columns_type_inverse' is not defined

In [72]:
        
            try:
                return int(x.strip())
            except:
                if x == "time\\geo":
                    return "var_" + "geo"
                else:
                    return "var_" + x
    

,var_unit,var_hlth_pb,var_age,var_sex,var_time\geo,var_EU28,var_BE,var_BG,var_CZ,var_DK,...,var_FI,var_SE,var_UK,var_IS,var_CH,var_TR,variable,value,value_raw,eurostat_annotation


In [17]:
result = tsv_to_dataframe_long("file.tsv.gz")
result

,var_unit,var_sex,var_age,var_agedef,var_geo,variable,value,value_raw,eurostat_annotation
0,NR,F,TOTAL,COMPLET,AM,2020,NaN,:,
1,NR,F,TOTAL,COMPLET,AT,2020,NaN,:,
2,NR,F,TOTAL,COMPLET,AZ,2020,NaN,:,
3,NR,F,TOTAL,COMPLET,BE,2020,0.31008,0.31008,
4,NR,F,TOTAL,COMPLET,BG,2020,0.50948,0.50948,
...,...,...,...,...,...,...,...,...,...
173037,NR,M,Y_GE60,COMPLET,SM,1990,NaN,:,
173038,NR,M,Y_GE60,COMPLET,TR,1990,NaN,:,
173039,NR,M,Y_GE60,COMPLET,UA,1990,NaN,:,
173040,NR,M,Y_GE60,COMPLET,UK,1990,NaN,:,


In [24]:
#Python library

import tsv_to_pandas

In [35]:
eurostat_link = "https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/t2020_rd300.tsv.gz"
urllib.request.urlretrieve(eurostat_link , "file.tsv.gz")

('file.tsv.gz', <http.client.HTTPMessage at 0x7fc68ddd8850>)

In [36]:
result = tsv_to_pandas.tsv_to_dataframe("file.tsv.gz")

In [37]:
result.columns[4]

'2003 '

In [38]:
result = tsv_to_pandas.tsv_to_dataframe_long("file.tsv.gz")
result

,var_geo,variable,value,value_raw,eurostat_annotation
0,AT,2000,10.2,10.2,
1,BE,2000,15.0,15.0,
2,BG,2000,7.0,7.0,
3,CH,2000,8.0,8.0,
4,CY,2000,13.2,13.2,
...,...,...,...,...,...
730,SE,2020,4.6,4.6,
731,SI,2020,7.6,7.6,
732,SK,2020,6.8,6.8,
733,TR,2020,6.4,6.4,


In [39]:
result.head(100)

,var_geo,variable,value,value_raw,eurostat_annotation
0,AT,2000,10.2,10.2,
1,BE,2000,15.0,15.0,
2,BG,2000,7.0,7.0,
3,CH,2000,8.0,8.0,
4,CY,2000,13.2,13.2,
...,...,...,...,...,...
95,NL,2002,14.0,14.0,NaN
96,NO,2002,12.3,12.3,NaN
97,PL,2002,10.1,10.1,NaN
98,PT,2002,8.4,8.4,NaN


In [40]:
print(result["variable"].max())

2020


In [45]:
import xml.etree.ElementTree as ET


tree = ET.parse('read.ttl')
root = tree.getroot()


ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [46]:
import rdflib

ModuleNotFoundError: No module named 'rdflib'

In [48]:
import xml

ERROR: Could not find a version that satisfies the requirement xml (from versions: none)
ERROR: No matching distribution found for xml
Note: you may need to restart the kernel to use updated packages.
